<a href="https://colab.research.google.com/github/Aayush-Keshwani/Suggestion_Model/blob/main/Suggesstionmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

class MovieRecommender:
    def __init__(self, data_path):
        with open(data_path, 'r') as f:
            data = json.load(f)

        self.movies = data['movies']
        self.genres = data['genres']

        # Prepare genre features
        self.mlb = MultiLabelBinarizer()
        genre_features = self.mlb.fit_transform([movie['genres'] for movie in self.movies])

        # Calculate similarity matrix
        self.similarity_matrix = cosine_similarity(genre_features)

        # Create movie title to index mapping
        self.title_to_index = {movie['title'].lower(): idx for idx, movie in enumerate(self.movies)}

    def recommend_by_movie(self, movie_title, top_n=5):
        """Recommend movies similar to the given movie title"""
        try:
            idx = self.title_to_index[movie_title.lower()]
        except KeyError:
            return {"error": f"Movie '{movie_title}' not found in database."}

        # Get similarity scores for this movie
        sim_scores = list(enumerate(self.similarity_matrix[idx]))

        # Sort by similarity score
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get top N most similar movies (skip the first which is the movie itself)
        sim_scores = sim_scores[1:top_n+1]

        # Get movie indices and similarity scores
        movie_indices = [i[0] for i in sim_scores]

        # Prepare output dictionary
        output = {
            "input_movie": movie_title,
            "recommendations": [
                {
                    "title": self.movies[i]['title'],
                    "year": self.movies[i]['year'],
                    "genres": self.movies[i]['genres'],
                    "similarity_score": float(sim_scores[j][1])  # Convert numpy float to Python float
                }
                for j, i in enumerate(movie_indices)
            ]
        }

        return output

    def recommend_by_genres(self, genres, top_n=5):
        """Recommend movies based on a list of genres"""
        # Convert input genres to binary vector
        input_genres = [g.lower() for g in genres]
        genre_vector = self.mlb.transform([input_genres])

        # Calculate similarity with all movies
        all_movies_features = self.mlb.transform([movie['genres'] for movie in self.movies])
        similarity_scores = cosine_similarity(genre_vector, all_movies_features)

        # Get top N most similar movies
        top_indices = np.argsort(similarity_scores[0])[::-1][:top_n]

        # Prepare output dictionary
        output = {
            "input_genres": genres,
            "recommendations": [
                {
                    "title": self.movies[i]['title'],
                    "year": self.movies[i]['year'],
                    "genres": self.movies[i]['genres'],
                    "similarity_score": float(similarity_scores[0][i])  # Convert numpy float to Python float
                }
                for i in top_indices
            ]
        }

        return output

    def get_random_movies(self, n=5):
        """Get random movies from the database"""
        indices = np.random.choice(len(self.movies), size=n, replace=False)

        # Prepare output dictionary
        output = {
            "random_movies": [
                {
                    "title": self.movies[i]['title'],
                    "year": self.movies[i]['year'],
                    "genres": self.movies[i]['genres']
                }
                for i in indices
            ]
        }

        return output

def write_to_json(data, filename):
    """Write data to a JSON file"""
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Example usage
if __name__ == "__main__":
    # Initialize the recommender with the JSON data
    recommender = MovieRecommender('/content/drive/MyDrive/db.json')

    # Generate recommendations
    inception_recs = recommender.recommend_by_movie("Inception")
    genre_recs = recommender.recommend_by_genres(['Action', 'Adventure', 'Sci-Fi'])
    random_movies = recommender.get_random_movies()

    # Combine all results into one dictionary
    all_recommendations = {
        "movie_based_recommendations": inception_recs,
        "genre_based_recommendations": genre_recs,
        "random_movies": random_movies
    }

    # Write to JSON file
    write_to_json(all_recommendations, 'movie_recommendations.json')

    print("Recommendations have been written to 'movie_recommendations.json'")

Recommendations have been written to 'movie_recommendations.json'


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['action', 'adventure', 'sci-fi'] will be ignored
  warnings.warn(
